# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [27]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [28]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [29]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2024-08-12,The landscape of ontologies in materials scien...,SeMatS 2024 (International Workshop on Semanti...,This paper provides an overview of ontologies ...,"E. Norouzi, J.Waitelonis, H. Sack. The landsca...",OE-MSE-SeMats-7,https://arxiv.org/abs/2408.06034
1,2024-04-29,OAEI Machine Learning Dataset for Online Model...,ESWC 2024 Poster,Ontology and knowledge graph matching systems ...,"Hertling, S., Norouzi, E., & Sack, H. (2024). ...",OAEI-ml-dataset-6,https://arxiv.org/abs/2404.18542
2,2023-03-21,Enhancing Entity Alignment Between Wikidata an...,International Joint Workshop on Semantic Web a...,This paper focuses on the Wikidata and A 𝑟𝑡 G ...,"Lippolis, Anna Sofia et al. “Enhancing Entity ...",EA-Artgraph-llm-5,https://api.semanticscholar.org/CorpusID:26391...
3,2023-09-07,Knowledge Graph Based RDM Solutions: NFDI4Cult...,1st Conference on Research Data Infrastructure...,Based on our experience within the NFDI4Cultur...,"Sack, H., Schrade, T., Bruns, O., Posthumus, E...",KG-base-RDM-cordi-4,https://doi.org/10.52825/cordi.v1i.371
4,2024-03-21,Data-Driven Accelerated Parameter Identificati...,Experimental Mechanics Journal,The objective of this paper is to assess the f...,"Morand, L., Norouzi, E., Weber, M. et al. Data...",parameter-identification-relaxation-exp-mech-1,https://doi.org/10.1007/s11340-024-01057-x
5,2021-05-14,In situ correlation between metastable phase-t...,Nature Communications,"In this work, the kinetic and chemical conditi...","Orava, J., Balachandran, S., Han, X. et al. In...",metallic-glass-nature-2,https://www.nature.com/articles/s41467-021-230...
6,2018-10-26,Optimizing Grain Selection Design in the Singl...,Crystal Research and Technology,An attempt is made to model the design of grai...,"F. Sadeghi, A. Kermanpur, E. Norouzi, Crystal ...",single-crystal-grain-3,https://doi.org/10.1002/crat.201800108


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [30]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [31]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w', encoding='utf-8') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [32]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [33]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
